## Backtesting using historical prices

In [ ]:
import datetime as dt
from datetime import timedelta
import numpy as np
import pandas as pd
from numpy import dot
from numpy import divide
from numpy.linalg import multi_dot as mdot
from numpy.linalg import inv
import matplotlib.pyplot as plt
from matplotlib import rc
import quadprog
import yfinance
import os
from weights import Portfolio
np.set_printoptions(precision = 3, suppress = True, linewidth = 400)
os.chdir('/Users/nielseriksen/thesis/')

### Downloading historical data

In [ ]:
tickers = ['IVV', 'HYG', 'GC=F', 'IYT']
etfs = yfinance.download(tickers, auto_adjust = True, start = "2011-9-1", end='2019-9-1')['Close']
#etfs = yfinance.download(tickers, start = "2011-9-1")['Close']

In [ ]:
etfs = etfs/etfs.iloc[0]
etfs = etfs.pct_change().iloc[1:]*100

In [ ]:
etfs.to_csv("data/etfs.csv", sep=";")
#etfs = pd.read_csv("../data/etfs.csv", sep=";", index_col = 0)

## Steps in GARCH backtesting

---- Inital setup----

$t_{-1}$ is last period/yesterday - all information about this period is known when in period $t$


$t$ is our current period/today - this is where weights are decided

$t_{+1}$ is next period/tomorrow - we attempt to forecast the volatility of this period


0. Fit model
1. Receive variables and parse them: sigma ($\sigma$), dcca (a), dccb (b), residuals ($\epsilon$), alpha ($\alpha$), beta ($\beta$), omega ($\omega$)
2. Calculate Qbar = $\frac{1}{T}\sum_{t=1}^T\eta_t\eta_t'$
---- Enter loop ---- 

0. Get variables from current period for all assets: $\epsilon^2_{t}$, $\sigma^2_{t}$
1. Calculate for all assets $\epsilon_t = r_t - \mu$
2. Calculate for all assets $ \sigma_{i,t+1}^2=\omega_i+\alpha_i\epsilon_{i,t}^2+\beta_i\sigma_{i,t}^2$
3. Calculate $Var_{t} = diag(\sigma_{t}^2)$
3. Calculate $Var_{t+1} = diag(\sigma_{t+1}^2)$
4. Calculate $\eta_t = inv(Var_t)*\epsilon_t$
5. Calculate $Q_{t+1} = Qbar*(1-dcca-dccb) + dcca*\eta_{t}*\eta_{t}' + b*Q_{t}$
6. Calculate $Q_{t+1}^{*-1} = diag(Q_{t+1}^{-1})$ 
7. Calculate $\Gamma_{t+1}=Q_{t+1}^{*-1}Q_{t+1}Q_{t+1}^{*-1}$
8. Calculate $\Omega_{t+1}=\text{Var}_{t+1}\Gamma_{t+1}\text{Var}_{t+1}$

## Obtain params from R

### Fitting initial model

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
pandas2ri.activate()
os.chdir('/Users/nielseriksen/thesis/')
# Defining the R script and loading the instance in Python
r = ro.r
r['source']('backtesting/fitting_mgarch.R')

### Initial setup

In [ ]:
# 0. Fit model
length_sample_period = len(etfs)-1000
out_of_sample = etfs.iloc[length_sample_period:,]
in_sample = etfs.iloc[:length_sample_period,]
# Loading the function we have defined in R.
fit_mgarch_r = ro.globalenv['fit_mgarch']
#Fitting the mgarch model and receiving the result
ugarch_model = "sGARCH"
ugarch_dist_model = "norm"
coef, residuals, sigmas = fit_mgarch_r(length_sample_period, ugarch_model, ugarch_dist_model)

In [ ]:
def calculate_Qbar(epsilons, sigmas):
    assert np.size(epsilons, axis=0) == np.size(sigmas, axis=0)
    eta = []
    p = np.size(epsilons, 1)
    for epsilon_t, sigma_t in zip(epsilons, sigmas):
        # Ensure epsilon_t is px1 dimensions so eta becomes px1
        epsilon_t = np.reshape(epsilon_t, (p, 1))
        Var_t, Var_t_inv = inv(calculate_Var_t(sigma_t))
        eta.append(dot(Var_t_inv, epsilon_t))

    eta = np.array(eta)
    Qbar = 1/len(epsilons) * sum([dot(eta, eta.T) for eta in eta])
    assert np.size(Qbar, 1) == np.size(epsilons, 1)
    return Qbar

In [ ]:
#Receive variables from model
asset_names = etfs.columns.values
p = len(asset_names)

# How elegant
mu, o, al, be = np.hsplit(coef[:-2].reshape((len(asset_names), 4)), 4)
mu, o, al, be = map(np.ravel, (mu, o, al, be))  # Flattening to 1d array
mu, o, al, be = map(np.reshape, (mu, o, al, be), [(p, 1)]*4)  # Reshaping to px1
dcca = coef[-2]
dccb = coef[-1]
epsilons = residuals
_sigmas = sigmas

# 5. Calculate Qbar
Qbar = calculate_Qbar(epsilons, sigmas)
Q_t = Qbar

### Enter loop

In [ ]:
Omega_ts = []
for t, r_t in enumerate(out_of_sample.values):
    r_t = np.reshape(r_t, (p, 1))
    # 0. Get current sigma^2
    s_t_squared = np.reshape(_sigmas[-1], (p, 1))

    # 1. Calculate current period epsilon
    e_t, e_t_squared, epsilons = calculate_epsilon(epsilons, r_t, mu)

    # 2. Calculate for all assets sigma_^2
    s_t_squared_plus_1 = calculate_sigma(o, al, e_t_squared, be, s_t_squared)

    # 3. Calculate Var_t, Var_t_inv
    Var_t, Var_t_inv = calculate_Var_t(s_t_squared)
    Var_t_plus_1, Var_t_plus_1_inv = calculate_Var_t(s_t_squared_plus_1)
    
    # 4. Calculate eta_t
    eta_t = dot(Var_t_inv, e_t)
    
    # 6. Calculate Q_t_plus_1
    Q_t_plus_1, Q_t_plus_1_s_inv = calculate_Q_t_plus_1(Qbar, dcca, dccb, eta_t, Q_t)
    
    # 8. Calulate Gamma_t
    Gamma_t_plus_1 = calculate_Gamma_t_plus_1(Q_t_plus_1_s_inv, Q_t_plus_1)
    
    # 9. 
    Omega_t_plus_1 = calculate_Omega_t_plus_1(Var_t_plus_1, Gamma_t_plus_1)
    
    # Storing
    Omega_ts.append(Omega_t_plus_1)
    _sigmas = np.append(_sigmas, np.reshape(s_t_squared, (1, p)), axis = 0)
    
    # Iterate on period
    Q_t = Q_t_plus_1


In [ ]:
def values_from_last_period(epsilons, sigmas):
    e_t = epsilons[-1]
    e_t_squared = epsilons[-1]**2
    s_t_squared = sigmas[-1]
    return e_t, e_t_squared, s_t_squared

In [ ]:
def calculate_epsilon(epsilons, r_t, mu):
    p = np.size(epsilons, 1)
    
    # Ensure e_t is px1
    e_t = np.reshape(np.array([r_t - mu]).T, (p, 1))
    e_t_squared = e_t**2
    epsilons = np.append(epsilons, e_t.T, axis=0)
    return e_t, e_t_squared, epsilons

In [ ]:
def calculate_sigma(o, al, e_t_1_squared, be, s_t_1_squared):
    
    next_sigma = np.array([o + al*e_t_1_squared + be*s_t_1_squared])
    return next_sigma

In [ ]:
def calculate_Var_t(s_t_squared):
    Var_t = np.diag(np.ravel(s_t_squared))
    Var_t_inv = inv(Var_t)
    return Var_t, Var_t_inv

In [ ]:
def calculate_Q_t_plus_1(Qbar, dcca, dccb, eta_t, Q_t):
    assert np.size(dcca) == 1
    assert np.size(dccb) == 1
    Q_t_plus_1 = np.array(Qbar*(1-dcca-dccb) + dcca*eta_t*eta_t.T + dccb*Q_t)
    #print(Q_t_plus_1)
    Q_t_plus_1_s_inv = inv(np.diag(np.diag(Q_t_plus_1)))
    return Q_t_plus_1, Q_t_plus_1_s_inv

In [ ]:
def calculate_Gamma_t_plus_1(Q_t_plus_1_s_inv, Q_t_plus_1):
    Gamma_t_plus_1 = mdot([Q_t_plus_1_s_inv, Q_t_plus_1, Q_t_plus_1_s_inv])
    return Gamma_t_plus_1

In [ ]:
def calculate_Omega_t_plus_1(Var_t_plus_1, Gamma_t_plus_1):
    Omega_t_plus_1 = mdot([Var_t_plus_1, Gamma_t_plus_1, Var_t_plus_1])
    return Omega_t_plus_1

### Generating weigths from this

In [ ]:
ones = np.ones((p, 1))
v_t = [np.ravel(divide(dot(Omega, ones), mdot([ones.T, Omega, ones]))) for Omega in Omega_ts]

In [ ]:
plt.plot(v_t)
#plt.legend(labels=["IVV", "HYG"])

In [ ]:
etfs.std()

In [ ]:
method_two = pd.DataFrame(v_t*out_of_sample.values).sum(axis=1)

In [ ]:
plt.plot(method_two)

In [ ]:
garch_strategy = (pd.DataFrame(v_t*out_of_sample.values).sum(axis=1)/100+1).cumprod()

In [ ]:
out_of_sample.mean(axis=1).values

In [ ]:
garch_strategy.std()

In [ ]:
one_over_n_strategy = (out_of_sample.mean(axis=1)/100 +1).cumprod()

In [ ]:
one_over_n_strategy.std()

In [ ]:
garch_strategy.index = one_over_n_strategy.index

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(garch_strategy, label = "stationary GARCH")
ax.plot(one_over_n_strategy, label = "$1/N$")
ax.legend()

### GARCH with trading costs

In [ ]:
dot(np.full((1, p), 0.5), Omega_ts[-1])

In [ ]:
Omega_ts[-1]**2

In [ ]:
def Avv(rho, lambda_, Omega_t_plus_1):
    r = rho
    l = lambda_
    O = Omega_t_plus_1
    
    res = np.sqrt(l*(O**2)+0.25*((rho**2)*((1-rho)**(-2))*(l*O)**2 + 2*rho*((1-rho)**(-1))*l*(O**2)+O**2))-0.5*(O+rho*((1-rho)**(-1))*l*O)
    return res
    
rho = 0.0001    
Omega_t_plus_1 = Omega_ts[-1]     # Picking a random Omega
lambda_ = 0.95
test_Avv = Avv(rho, lambda_, Omega_t_plus_1)

## Testing $v_t= v_{t-1}(1+\Lambda^{-1}A_{vv})$

In [ ]:
v_t_minus_1 = np.full((p, 1), 1/p)

In [ ]:
dot((1/(lambda_*Omega_t_plus_1)), test_Avv)

In [ ]:
v_t = dot(v_t_minus_1.T, (1+dot((1/(lambda_*Omega_t_plus_1)), test_Avv)))

In [ ]:
v_t = v_t/v_t.sum()
v_t = v_t.reshape((p, 1))

In [ ]:
mdot([v_t.T, Omega_t_plus_1, v_t])

In [ ]:
mdot([v_t_minus_1.T, Omega_t_plus_1, v_t_minus_1])

In [ ]:
Lambda_inv = (1/(lambda_*Omega_t_plus_1))

In [ ]:
aim = dot(inv(test_Avv), np.ones((p, 1)))/mdot([np.ones((p, 1)).T, inv(test_Avv), np.ones((p, 1))])
aim = dot(inv(test_Avv), np.ones((p, 1)))

In [ ]:
aim

In [ ]:
v_t = v_t_minus_1 + dot(dot(Lambda_inv, test_Avv), (aim - v_t_minus_1))

In [ ]:
v_t

In [ ]:
test = np.array([1, 2, 5, -3])


### Misc 

In [ ]:
rcov_forecast_r = ro.globalenv['rcov_forecast']
rcov = rcov_forecast_r()[0][0]

In [ ]:
tri = np.zeros((10, 10))
tri[np.triu_indices(n = 10, k=0, m=10)] = rcov

In [ ]:
Omega = tri + tri.T - np.diag(np.diag(tri))